# Assignment a02
### A. Thieshanthan, 180641N

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import cv2 as cv
import string
from os import path

def plot_square(P, ax, color = '#6699cc'):
    P=P/P[-1, :]
    P=np.insert(P, 4, P[:,0], axis=1)
    x=P[0, :]
    y=P[1, :]
    ax.plot(x, y, color = color, alpha=0.7, linewidth=3, solid_capstyle='round', zorder=2)
    ax.set_aspect('equal')

def save_img(img, name):
    dir = 'outputs/' + name + '.jpg'
    cv.imwrite(dir, img)

## 1. Using the code given to experimenting various types of 2D transformations

In [3]:
# points
a, b, c, d = (0,0,1), (0,1,1), (1.5,.5,1), (1,0,1)
P = np.array([a,b,c,d]).T


t = np.pi/3
H = [[np.cos(t), np.sin(t),0.],
    [-np.sin(t), np.cos(t),0.], 
    [0.,0.,1.]]

Pt = np.matmul(H,P)

t = 0
H1 = [[np.cos(t), np.sin(t),1.],
    [-np.sin(t), np.cos(t),1.], 
    [0.,0.,1]]
Pt1 = np.matmul(H1, P)

t = 0
H2 = [[np.cos(t), np.sin(t),0.],
    [-np.sin(t), np.cos(t),0.], 
    [0.,0.,0.4]]
Pt2 = np.matmul(H2, P)

fig, ax = plt.subplots(1,1, sharex=True, sharey=True)
plot_square(P, ax, 'b')         #original square - blue
plot_square(Pt, ax, 'y')        #Yellow - rotated 60 degrees
plot_square(Pt1, ax, 'r')       #Red - tranlated
plot_square(Pt2, ax, 'c')       #cyan - scaled up

plt.savefig('outputs/2dtransforms')

FileNotFoundError: [Errno 2] No such file or directory: 'outputs/2dtransforms.png'

## 2. Transforming graffiti images using provided code

In [1]:
im1 = cv.imread('images/img1.ppm', cv.IMREAD_ANYCOLOR)
im5 = cv.imread('images/img5.ppm', cv.IMREAD_ANYCOLOR)

save_img(im1, 'im1')
save_img(im5, 'im5')

print(im1.shape, im5.shape)

NameError: name 'cv' is not defined

In [5]:
def warp_image(img, H, shape = (2000,2000,3)):
    m,n,ch = shape #canvas size
    indy, indx = np.indices((m, n), dtype=np.float32)
    lin_homg_ind = np.array([indx.ravel(), indy.ravel(), np.ones_like(indx).ravel()])
    #warping
    map_ind = np.matmul(H, lin_homg_ind)
    x, y = map_ind[:-1]/map_ind[-1]
    x = x.reshape(m,n).astype(np.float32)
    y = y.reshape(m,n).astype(np.float32)
    warped_img = cv.remap(img, x, y, cv.INTER_LINEAR)
    
    return warped_img

In [1]:
def stitch(img1, img2, pad = (200, 200)):
    '''Stitch img1 onto img2
	img2 - reference image
	'''
    m,n,c = img2.shape
    m += pad[0]
    n += pad[1]
    canvas = np.zeros((m, n, c))
    canvas[0:img2.shape[0], 0:img2.shape[1]] = img2
    for i in range(m):
        for j in range(n):
             if np.all(img1[i][j]) != 0:
                 canvas[i,j] = img1[i,j]
    return canvas

KeyboardInterrupt: 

In [36]:
# Image 1 to image 5
with open('images/H1to5p') as f:
    H = [[float(x) for x in line.split()] for line in f]
print(H)
H = np.array(H)

im1_warped = warp_image(im1, np.linalg.inv(H), (1200,1200,3))
im15 = stitch(im1_warped, im5, pad = 100)

save_img(im1, 'im1')
save_img(im5, 'im5')
save_img(im1_warped,'image1warped')
save_img(im15, 'im1to5')


[[0.62544644, 0.057759174, 222.01217], [0.22240536, 1.1652147, -25.605611], [0.00049212545, -3.6542424e-05, 1.0]]


## 3. Stiching by selecting points from mouse click and using opencv functions

In [37]:
N = 5     #Number of points to be selected
global n
n = 0
p1 = np.empty((N,2))        #selected points on first image 
p2 = np.empty((N,2))        # selected points on second image 

#mouse callback function
def draw_circle(event, x, y, flags, param):
    global n 
    p = param[0]
    if event == cv.EVENT_LBUTTONDOWN:
        cv.circle(param[1], (x,y), 5, (255,0,0), -1)    #draws a circle on clicked point
        p[n] = (x,y)
        n += 1


im1copy = im1.copy()
im5copy = im5.copy()


cv.namedWindow('Image 1', cv.WINDOW_AUTOSIZE)
param = [p1, im1copy]
cv.setMouseCallback('Image 1', draw_circle, param)

while(1):
    cv.imshow('Image 1', im1copy)
    if n == N:
        break
    if cv.waitKey(20) & 0xFF == 27:
        break

cv.namedWindow('Image 5', cv.WINDOW_AUTOSIZE)
param = [p2, im5copy]
cv.setMouseCallback('Image 5', draw_circle, param)
n = 0
while(1):
    cv.imshow('Image 5', im5copy)
    if n == N:
        break
    if cv.waitKey(20) & 0xFF == 27:
        break
cv.destroyAllWindows()


[[ 6.67833889e-01  4.74546565e-02  2.19691891e+02]
 [ 2.24630816e-01  1.15494975e+00 -1.97995354e+01]
 [ 5.54027726e-04 -6.70285922e-05  1.00000000e+00]]
[[ 6.20252362e-01  3.39448550e-02  2.21593331e+02]
 [ 2.34315085e-01  1.09731364e+00 -2.45618192e+01]
 [ 4.95386157e-04 -1.29241854e-04  1.00000000e+00]]


IndexError: index 640 is out of bounds for axis 0 with size 640

In [41]:
H1, status = cv.findHomography(p1,p2)

H= np.array(H, dtype = np.float32)
print(H1)

im1_m1 = warp_image(im1, np.linalg.inv(H1), (1200,1200,3))
im15_m1 = stitch(im1_m1, im5)

cv.imwrite('outputs/im15m1.png', im15_m1)
print(np.sum((H-H1)**2))

[[ 6.67833889e-01  4.74546565e-02  2.19691891e+02]
 [ 2.24630816e-01  1.15494975e+00 -1.97995354e+01]
 [ 5.54027726e-04 -6.70285922e-05  1.00000000e+00]]
39.09625971079854


## 4. Stitching images by selecting points from mouse click and computing homograpy with own code

In [43]:
# p1, p2 are use here as well
A = []

for i in range(N):
    x, y = p1[i][0], p1[i][1]
    u, v = p2[i][0], p2[i][1]
    A.append([x, y, 1, 0, 0, 0, -u*x, -u*y, -u])
    A.append([0, 0, 0, x, y, 1, -v*x, -v*y, -v])
A = np.asarray(A)
U, S, Vh = np.linalg.svd(A)
L = Vh[-1,:] / Vh[-1,-1]
h = L.reshape(3, 3)

print(h)
print(np.sum((H-h)**2))

im1m3= cv.warpPerspective(im1, h, (1200,1200))
im15m3 = stitch(im1m3, im5)
cv.imwrite('outputs/im15m3.png', im15m3)


[[ 6.67668288e-01  4.71601809e-02  2.19492779e+02]
 [ 2.24284933e-01  1.15325819e+00 -1.95866380e+01]
 [ 5.53305900e-04 -6.90841212e-05  1.00000000e+00]]
42.57744920921803


True

## 5. Stitchong using superglue


In [68]:
!git clone https://github.com/magicleap/SuperGluePretrainedNetwork.git

Cloning into 'SuperGluePretrainedNetwork'...
Updating files: 100% (89/89)
Updating files: 100% (89/89), done.


In [116]:
!python SuperGluePretrainedNetwork/match_pairs.py --viz --resize -1

Namespace(cache=False, eval=False, fast_viz=False, force_cpu=False, input_dir='outputs/', input_pairs='SuperGluePretrainedNetwork/assets/match.txt', keypoint_threshold=0.005, match_threshold=0.2, max_keypoints=1024, max_length=-1, nms_radius=4, opencv_display=False, output_dir='outputs/matches/', resize=[-1], resize_float=False, show_keypoints=False, shuffle=False, sinkhorn_iterations=20, superglue='indoor', viz=True, viz_extension='png')
Will not resize images
Running inference on device "cpu"
Loaded SuperPoint model
Loaded SuperGlue model ("indoor" weights)
Looking for data in directory "outputs"
Will write matches to directory "outputs\matches"
Will write visualization images to directory "outputs\matches"
[Finished pair     0 of     1] load_image=0.089 matcher=6.756 viz_match=0.722 total=7.567 sec {0.1 FPS} 
d:\Semester 4\EN2550_Fundamentals of Image Processing and Machine Vision\Assingments\assignment2\SuperGluePretrainedNetwork\models\superpoint.py:171: UserWarning: This overload

In [45]:
mat = np.load('outputs/matches/im1_im5_matches.npz')
print(mat.files)
print(mat['keypoints0'].shape)
print(mat['matches'][2])
print(np.sum(mat['matches'] > -1))
print(mat['keypoints0'][0])

['keypoints0', 'keypoints1', 'matches', 'match_confidence']
(1024, 2)
-1
310
[ 79. 500.]


In [49]:
q1 = []
q2 = []
for i in range(len(mat['matches'])):
    if mat['matches'][i] !=-1:
        q1.append(mat['keypoints0'][i])
        q2.append(mat['keypoints1'][mat['matches'][i]])
q1 = np.asarray(q1)
q2 = np.asarray(q2)
print(q1.shape)

(310, 2)


In [50]:
h4, mask = cv.findHomography(q1, q2 , method = cv.RANSAC)
print(H,'\n', h4)
print(np.sum((H-h4)**2))
im1ran = cv.warpPerspective(im1, h4, (1200,1200))
im15su = stitch(im1ran, im5)
cv.imwrite('outputs/im15su.png', im15su)

[[ 6.2544644e-01  5.7759173e-02  2.2201218e+02]
 [ 2.2240536e-01  1.1652147e+00 -2.5605612e+01]
 [ 4.9212546e-04 -3.6542424e-05  1.0000000e+00]] 
 [[ 6.17522839e-01  5.43293411e-02  2.22337085e+02]
 [ 2.16388403e-01  1.15471166e+00 -2.24786917e+01]
 [ 4.74669659e-04 -4.16988122e-05  1.00000000e+00]]
9.883415981580645


True

### Stitching multiple images

In [143]:
!python SuperGluePretrainedNetwork/match_pairs.py --viz --resize -1 --input_dir images

Namespace(cache=False, eval=False, fast_viz=False, force_cpu=False, input_dir='images', input_pairs='SuperGluePretrainedNetwork/assets/match.txt', keypoint_threshold=0.005, match_threshold=0.2, max_keypoints=1024, max_length=-1, nms_radius=4, opencv_display=False, output_dir='outputs/matches/', resize=[-1], resize_float=False, show_keypoints=False, shuffle=False, sinkhorn_iterations=20, superglue='indoor', viz=True, viz_extension='png')
Will not resize images
Running inference on device "cpu"
Loaded SuperPoint model
Loaded SuperGlue model ("indoor" weights)
Looking for data in directory "images"
Will write matches to directory "outputs\matches"
Will write visualization images to directory "outputs\matches"
[Finished pair     0 of     7] load_image=0.008 matcher=4.178 viz_match=0.805 total=4.991 sec {0.2 FPS} 
[Finished pair     1 of     7] load_image=0.008 matcher=4.195 viz_match=0.743 total=4.946 sec {0.2 FPS} 
[Finished pair     2 of     7] load_image=0.008 matcher=4.297 viz_match=0.

In [51]:
# loading outputs from superglue
m1 = np.load('outputs/matches/img1_img2_matches.npz')
m2 = np.load('outputs/matches/img2_img3_matches.npz')
m3 = np.load('outputs/matches/img3_img4_matches.npz')
matches = [m1, m2, m3]
number_of_matches = {}
for i in range(len(matches)):
    number_of_matches['m' + str(i)] = np.sum(matches[i]['matches'] > -1)
print(number_of_matches)


{'m0': 454, 'm1': 257, 'm2': 227}


In [63]:
# Images for multiple image stitching
img1 = cv.imread('images\img1.jpg', cv.IMREAD_COLOR)
img2 = cv.imread('images\img2.jpg', cv.IMREAD_COLOR)
img3 = cv.imread('images\img3.jpg', cv.IMREAD_COLOR)
img4 = cv.imread('images\img4.jpg', cv.IMREAD_COLOR)


## Image order to be stiched
(((1->2)->3)) ->4

In [55]:
def find_matching_points(m):
    q1 = []
    q2 = []
    for i in range(len(m['matches'])):
        if mat['matches'][i] !=-1:
            q1.append(m['keypoints0'][i])
            q2.append(m['keypoints1'][m['matches'][i]])
    q1 = np.asarray(q1)
    q2 = np.asarray(q2)
    return q1, q2

In [138]:
# stitching 1,2 
points12 = find_matching_points(m1)

h12, s= cv.findHomography(points12[0], points12[1], method= cv.RANSAC)


In [120]:
def stitch_left(img1, img2, pad = (1000, 1000)):
    '''Stitch img1 onto img2'''
    m,n,c = img2.shape
    m += pad[0]
    n += pad[1]
    canvas = np.zeros((m, n, c))
    canvas[pad[0]:img2.shape[0]+pad[0], pad[1]:img2.shape[1]+pad[1]] = img2
    for i in range(m):
        for j in range(n):
             if np.all(img1[i][j]) != 0:
                 canvas[i,j] = img1[i,j]
    return canvas

In [139]:
# H23 = np.matmul(h12, h23)
mount2 = warp_image(img2, (h12), (2000,2000,3))
mount2 = stitch(mount2, img1, pad = (50,350))

save_img(mount2, 'mount21')

In [130]:
!python SuperGluePretrainedNetwork/match_pairs.py --viz --resize -1 --input_dir outputs/

Namespace(cache=False, eval=False, fast_viz=False, force_cpu=False, input_dir='outputs/', input_pairs='SuperGluePretrainedNetwork/assets/match.txt', keypoint_threshold=0.005, match_threshold=0.2, max_keypoints=1024, max_length=-1, nms_radius=4, opencv_display=False, output_dir='outputs/matches/', resize=[-1], resize_float=False, show_keypoints=False, shuffle=False, sinkhorn_iterations=20, superglue='indoor', viz=True, viz_extension='png')
Will not resize images
Running inference on device "cpu"
Loaded SuperPoint model
Loaded SuperGlue model ("indoor" weights)
Looking for data in directory "outputs"
Will write matches to directory "outputs\matches"
Will write visualization images to directory "outputs\matches"
[Finished pair     0 of     1] load_image=0.018 matcher=5.701 viz_match=0.788 total=6.507 sec {0.2 FPS} 
d:\Semester 4\EN2550_Fundamentals of Image Processing and Machine Vision\Assingments\assignment2\SuperGluePretrainedNetwork\models\superpoint.py:171: UserWarning: This overload

In [137]:
m = np.load('outputs/matches/mount21_img3_matches.npz')
pts = find_matching_points(m)
print(pts[1].shape)
hf, s= cv.findHomography(pts[0], pts[1], method = cv.RANSAC)
print(hf)
mount = cv.warpPerspective(img3, hf, (1000,1000))
save_img(mount, 'mount_temp')
# mount = stitch(mount, mount2, pad = (1000,1000))

# save_img(mount, 'mount')

(307, 2)
[[-1.02971717e+00  1.55639253e+00  4.93011999e+02]
 [-9.83733552e-01  1.48691764e+00  4.71012704e+02]
 [-2.08822585e-03  3.15657553e-03  1.00000000e+00]]
